In [2]:
import sys
sys.path.append('..')
import os
import pandas as pd
from datetime import datetime, timedelta
from binance.client import Client
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("BINANCE_API_KEY")
api_secret = os.getenv("BINANCE_SECRET_KEY")
    
client = Client(api_key=api_key, api_secret=api_secret)

In [4]:
def write_klines_to_csv(symbol_, interval_, year):
    base_path = f'{symbol_}/{year}'
    os.makedirs(base_path, exist_ok=True)

    for month in range(12, 13):
        # Начало месяца
        month_start = datetime(year, month, 1)

        # Следующий месяц (для вычисления конца текущего)
        if month == 12:
            next_month_start = datetime(year + 1, 1, 1)
        else:
            next_month_start = datetime(year, month + 1, 1)

        total_days = (next_month_start - month_start).days
        chunk_days = total_days // 3

        for part in range(3):
            part_start = month_start + timedelta(days=chunk_days * part)
            part_end = month_start + timedelta(days=chunk_days * (part + 1)) if part < 2 else next_month_start
            print(part_start)
            print(part_end)
            file_name = f'{base_path}/{year}-{str(month).zfill(2)}-{part+1}.csv'
            print(file_name)
            if os.path.exists(file_name):
                continue

            print(f"Fetching: {file_name} | {part_start} -> {part_end}")

            klines = client.get_historical_klines(
                symbol=symbol_,
                interval=interval_,
                start_str=part_start.strftime('%Y-%m-%d'),
                end_str=part_end.strftime('%Y-%m-%d')
            )

            if not klines:
                print(f"No data for {file_name}")
                continue

            df = pd.DataFrame(klines).iloc[:, :6]
            df['timestamp'] = df[0]
            df['timestamp_human'] = pd.to_datetime(df[0], unit='ms')
            df['open'] = df[1].astype(float)
            df['high'] = df[2].astype(float)
            df['low'] = df[3].astype(float)
            df['close'] = df[4].astype(float)
            df['volume'] = (df[5].astype(float) * df[4].astype(float)).astype(int)
            df = df.drop(columns=[0, 1, 2, 3, 4, 5])
            df.to_csv(file_name)
            print(f"Saved: {file_name}")

write_klines_to_csv(symbol_='ETHUSDT', interval_='1s', year=2024)

2024-12-01 00:00:00
2024-12-11 00:00:00
ETHUSDT/2024/2024-12-1.csv
Fetching: ETHUSDT/2024/2024-12-1.csv | 2024-12-01 00:00:00 -> 2024-12-11 00:00:00
Saved: ETHUSDT/2024/2024-12-1.csv
2024-12-11 00:00:00
2024-12-21 00:00:00
ETHUSDT/2024/2024-12-2.csv
Fetching: ETHUSDT/2024/2024-12-2.csv | 2024-12-11 00:00:00 -> 2024-12-21 00:00:00
Saved: ETHUSDT/2024/2024-12-2.csv
2024-12-21 00:00:00
2025-01-01 00:00:00
ETHUSDT/2024/2024-12-3.csv
Fetching: ETHUSDT/2024/2024-12-3.csv | 2024-12-21 00:00:00 -> 2025-01-01 00:00:00
Saved: ETHUSDT/2024/2024-12-3.csv


In [11]:
write_klines_to_csv('ETHUSDT', '1s', 2022)

2022-01-01
2022-02-01

ETHUSDT/2022-01.csv
2022-02-01
2022-03-01

ETHUSDT/2022-02.csv
2022-03-01
2022-04-01

ETHUSDT/2022-03.csv
2022-04-01
2022-05-01

ETHUSDT/2022-04.csv
2022-05-01
2022-06-01

ETHUSDT/2022-05.csv
2022-06-01
2022-07-01

ETHUSDT/2022-06.csv
2022-07-01
2022-08-01

ETHUSDT/2022-07.csv
2022-08-01
2022-09-01

ETHUSDT/2022-08.csv
2022-09-01
2022-10-01

ETHUSDT/2022-09.csv
2022-10-01
2022-11-01

ETHUSDT/2022-10.csv
2022-11-01
2022-12-01

ETHUSDT/2022-11.csv
2022-12-01
2023-01-01

ETHUSDT/2022-12.csv
